In [1]:
import pandas as pd
import sqlite3

# 1. Load your cleaned dataset
df = pd.read_csv("cleaned_superstore.csv")   # file should be in the same folder as notebook
print("Dataset loaded. Shape:", df.shape)
print("Columns:", df.columns.tolist())

# 2. Create in-memory SQLite DB
conn = sqlite3.connect(":memory:")

# 3. Load dataframe into SQL table
df.to_sql("sales", conn, index=False, if_exists="replace")
print("Table 'sales' created in SQLite memory database.")

# 4. Define queries
queries = {
    "Profit margin by category": """
        SELECT Category, SUM(Profit)*1.0 / SUM(Sales) AS ProfitMargin
        FROM sales
        GROUP BY Category;
    """,
    "Profit margin by region": """
        SELECT Region, SUM(Profit)*1.0 / SUM(Sales) AS ProfitMargin
        FROM sales
        GROUP BY Region;
    """,
    "Top 5 profit-draining subcategories": """
        SELECT "Sub-Category", SUM(Profit) AS TotalProfit
        FROM sales
        GROUP BY "Sub-Category"
        ORDER BY TotalProfit ASC
        LIMIT 5;
    """,
    "Sales & profit by year": """
        SELECT Year, SUM(Sales) AS TotalSales, SUM(Profit) AS TotalProfit
        FROM sales
        GROUP BY Year
        ORDER BY Year;
    """,
    "Sales by customer segment": """
        SELECT Segment, SUM(Sales) AS TotalSales, SUM(Profit) AS TotalProfit
        FROM sales
        GROUP BY Segment;
    """
}

# 5. Run queries and display results
for title, query in queries.items():
    print(f"\n--- {title} ---")
    result = pd.read_sql(query, conn)
    print(result)


Dataset loaded. Shape: (100, 16)
Columns: ['Order ID', 'Order Date', 'Region', 'Category', 'Sub-Category', 'Product Name', 'Sales', 'Profit', 'Quantity', 'Discount', 'Customer Name', 'Segment', 'Ship Mode', 'Year', 'Month', 'Quarter']
Table 'sales' created in SQLite memory database.

--- Profit margin by category ---
          Category  ProfitMargin
0        Furniture      0.054224
1  Office Supplies      0.035577
2       Technology      0.074452

--- Profit margin by region ---
    Region  ProfitMargin
0  Central      0.059973
1     East      0.078688
2    South      0.057713
3     West      0.036685

--- Top 5 profit-draining subcategories ---
  Sub-Category  TotalProfit
0      Binders       -89.47
1       Labels        31.30
2       Tables       106.72
3  Accessories       154.13
4      Storage       184.34

--- Sales & profit by year ---
   Year  TotalSales  TotalProfit
0  2023    90472.07       5085.1

--- Sales by customer segment ---
       Segment  TotalSales  TotalProfit
0    

In [2]:
pd.read_sql("""
    SELECT Category, SUM(Sales) AS TotalSales
    FROM sales
    GROUP BY Category
    ORDER BY TotalSales DESC;
""", conn)


,Category,TotalSales
0,Furniture,33080.66
1,Technology,32141.69
2,Office Supplies,25249.72


In [4]:
import os

# Create a folder for outputs
os.makedirs("query_outputs", exist_ok=True)

for title, query in queries.items():
    result = pd.read_sql(query, conn)
    
    # Clean the title to make a valid filename
    filename = title.replace(" ", "_").replace("&", "and").replace("-", "_").lower() + ".csv"
    filepath = os.path.join("query_outputs", filename)
    
    # Save result as CSV
    result.to_csv(filepath, index=False)
    print(f"✅ Saved: {filepath}")


✅ Saved: query_outputs\profit_margin_by_category.csv
✅ Saved: query_outputs\profit_margin_by_region.csv
✅ Saved: query_outputs\top_5_profit_draining_subcategories.csv
✅ Saved: query_outputs\sales_and_profit_by_year.csv
✅ Saved: query_outputs\sales_by_customer_segment.csv


In [9]:
with pd.ExcelWriter("query_outputs/superstore_analysis.xlsx") as writer:
    for title, query in queries.items():
        result = pd.read_sql(query, conn)
        sheet_name = title[:30]  # Excel sheet names must be ≤ 31 chars
        result.to_excel(writer, sheet_name=sheet_name, index=False)

print("✅ Saved all results into query_outputs/superstore_analysis.xlsx")
from IPython.display import FileLink

# This will show a clickable download link in your notebook
FileLink("query_outputs/superstore_analysis.xlsx")


✅ Saved all results into query_outputs/superstore_analysis.xlsx


C:\Users\HP\Music\query_outputs\superstore_analysis.xlsx

In [ ]:
import matplotlib.pyplot as plt

# Profit margin by category
query = """
SELECT Category, SUM(Profit)*1.0 / SUM(Sales) AS ProfitMargin
FROM sales
GROUP BY Category;
"""
df_category = pd.read_sql(query, conn)

plt.figure(figsize=(6,4))
plt.bar(df_category["Category"], df_category["ProfitMargin"])
plt.title("Profit Margin by Category")
plt.ylabel("Profit Margin")
plt.show()


# Sales & Profit by Year
query = """
SELECT Year, SUM(Sales) AS TotalSales, SUM(Profit) AS TotalProfit
FROM sales
GROUP BY Year
ORDER BY Year;
"""
df_year = pd.read_sql(query, conn)

plt.figure(figsize=(6,4))
plt.plot(df_year["Year"], df_year["TotalSales"], marker="o", label="Sales")
plt.plot(df_year["Year"], df_year["TotalProfit"], marker="o", label="Profit")
plt.title("Sales & Profit by Year")
plt.xlabel("Year")
plt.ylabel("Amount")
plt.legend()
plt.show()
